In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [58]:
# train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [ ]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [59]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [63]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [64]:
columns = []
for p in ["", "P1_"]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [21]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [22]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [67]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [23]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [24]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [41]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [75]:

train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=Y_train)

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=Y_val)

In [76]:
cat_model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=7000
)
cat_model.fit(train_data,eval_set=val_data)



0:	learn: 88409.6079413	test: 83188.7795327	best: 83188.7795327 (0)	total: 14ms	remaining: 1m 38s
1:	learn: 87477.7508423	test: 82036.9238833	best: 82036.9238833 (1)	total: 23.6ms	remaining: 1m 22s
2:	learn: 86495.7129906	test: 80726.0762524	best: 80726.0762524 (2)	total: 35ms	remaining: 1m 21s
3:	learn: 85592.0111893	test: 79747.6687287	best: 79747.6687287 (3)	total: 54.5ms	remaining: 1m 35s
4:	learn: 84666.3932741	test: 78450.1314636	best: 78450.1314636 (4)	total: 62.9ms	remaining: 1m 27s
5:	learn: 83839.6608975	test: 77238.9144933	best: 77238.9144933 (5)	total: 70.7ms	remaining: 1m 22s
6:	learn: 83161.8511738	test: 76246.3727983	best: 76246.3727983 (6)	total: 89.1ms	remaining: 1m 29s
7:	learn: 82419.9419009	test: 75316.2277213	best: 75316.2277213 (7)	total: 97.9ms	remaining: 1m 25s
8:	learn: 81666.8664160	test: 74127.3549234	best: 74127.3549234 (8)	total: 107ms	remaining: 1m 23s
9:	learn: 81015.1304002	test: 73222.3625164	best: 73222.3625164 (9)	total: 123ms	remaining: 1m 26s
10:	le

89:	learn: 58381.0217064	test: 38099.4446462	best: 38099.4446462 (89)	total: 1.24s	remaining: 1m 34s
90:	learn: 58305.8115995	test: 38008.2753659	best: 38008.2753659 (90)	total: 1.25s	remaining: 1m 35s
91:	learn: 58258.6557113	test: 37947.1608979	best: 37947.1608979 (91)	total: 1.26s	remaining: 1m 34s
92:	learn: 58175.4034050	test: 37801.8528477	best: 37801.8528477 (92)	total: 1.27s	remaining: 1m 34s
93:	learn: 58041.7632989	test: 37599.2065938	best: 37599.2065938 (93)	total: 1.29s	remaining: 1m 34s
94:	learn: 57855.7894100	test: 37539.0502624	best: 37539.0502624 (94)	total: 1.31s	remaining: 1m 35s
95:	learn: 57800.8324084	test: 37419.8562658	best: 37419.8562658 (95)	total: 1.32s	remaining: 1m 35s
96:	learn: 57418.3037091	test: 37365.7046947	best: 37365.7046947 (96)	total: 1.34s	remaining: 1m 35s
97:	learn: 57317.3557621	test: 37181.2716321	best: 37181.2716321 (97)	total: 1.35s	remaining: 1m 35s
98:	learn: 57241.3280059	test: 37035.5099309	best: 37035.5099309 (98)	total: 1.36s	remainin

171:	learn: 50765.4009377	test: 33273.7017094	best: 33273.7017094 (171)	total: 2.24s	remaining: 1m 29s
172:	learn: 50708.9363673	test: 33207.1914057	best: 33207.1914057 (172)	total: 2.26s	remaining: 1m 29s
173:	learn: 50700.8897451	test: 33100.3853082	best: 33100.3853082 (173)	total: 2.27s	remaining: 1m 28s
174:	learn: 50650.7970058	test: 33082.0855218	best: 33082.0855218 (174)	total: 2.27s	remaining: 1m 28s
175:	learn: 50610.9250998	test: 33027.6713869	best: 33027.6713869 (175)	total: 2.29s	remaining: 1m 28s
176:	learn: 50554.1746852	test: 33023.2078124	best: 33023.2078124 (176)	total: 2.3s	remaining: 1m 28s
177:	learn: 50524.6325140	test: 32974.1656880	best: 32974.1656880 (177)	total: 2.31s	remaining: 1m 28s
178:	learn: 50317.6933861	test: 32950.4888344	best: 32950.4888344 (178)	total: 2.32s	remaining: 1m 28s
179:	learn: 50209.5434630	test: 32944.0137238	best: 32944.0137238 (179)	total: 2.33s	remaining: 1m 28s
180:	learn: 50174.8630643	test: 32926.1963692	best: 32926.1963692 (180)	to

254:	learn: 45787.2909537	test: 30829.6895392	best: 30829.6895392 (254)	total: 3.27s	remaining: 1m 26s
255:	learn: 45748.5377898	test: 30794.9000947	best: 30794.9000947 (255)	total: 3.3s	remaining: 1m 26s
256:	learn: 45741.9498661	test: 30767.6180433	best: 30767.6180433 (256)	total: 3.32s	remaining: 1m 27s
257:	learn: 45717.8489346	test: 30709.3811376	best: 30709.3811376 (257)	total: 3.34s	remaining: 1m 27s
258:	learn: 45598.2602239	test: 30704.0786710	best: 30704.0786710 (258)	total: 3.36s	remaining: 1m 27s
259:	learn: 45480.9805252	test: 30687.9051943	best: 30687.9051943 (259)	total: 3.38s	remaining: 1m 27s
260:	learn: 45409.8766973	test: 30672.7879861	best: 30672.7879861 (260)	total: 3.39s	remaining: 1m 27s
261:	learn: 45403.2757436	test: 30647.9424999	best: 30647.9424999 (261)	total: 3.4s	remaining: 1m 27s
262:	learn: 45172.8462428	test: 30611.2693133	best: 30611.2693133 (262)	total: 3.41s	remaining: 1m 27s
263:	learn: 44927.8270696	test: 30567.9322694	best: 30567.9322694 (263)	tot

347:	learn: 40255.2695706	test: 27457.4267533	best: 27457.4267533 (347)	total: 4.68s	remaining: 1m 29s
348:	learn: 40133.6353853	test: 27445.8852941	best: 27445.8852941 (348)	total: 4.69s	remaining: 1m 29s
349:	learn: 40093.5932093	test: 27403.4256630	best: 27403.4256630 (349)	total: 4.71s	remaining: 1m 29s
350:	learn: 40082.7020831	test: 27362.4826394	best: 27362.4826394 (350)	total: 4.73s	remaining: 1m 29s
351:	learn: 40063.4724788	test: 27299.9190156	best: 27299.9190156 (351)	total: 4.75s	remaining: 1m 29s
352:	learn: 40056.6407420	test: 27282.6040378	best: 27282.6040378 (352)	total: 4.76s	remaining: 1m 29s
353:	learn: 39809.2895864	test: 27271.2627912	best: 27271.2627912 (353)	total: 4.78s	remaining: 1m 29s
354:	learn: 39791.8948617	test: 27254.3668628	best: 27254.3668628 (354)	total: 4.79s	remaining: 1m 29s
355:	learn: 39778.5116076	test: 27155.1354388	best: 27155.1354388 (355)	total: 4.8s	remaining: 1m 29s
356:	learn: 39687.1889437	test: 27127.1896839	best: 27127.1896839 (356)	to

...

6617:	learn: 3235.4929133	test: 4427.2392367	best: 4427.2392367 (6617)	total: 1m 32s	remaining: 5.37s
6618:	learn: 3235.4580496	test: 4427.1445691	best: 4427.1445691 (6618)	total: 1m 32s	remaining: 5.35s
6619:	learn: 3234.7568630	test: 4426.7988501	best: 4426.7988501 (6619)	total: 1m 32s	remaining: 5.34s
6620:	learn: 3233.5191019	test: 4425.8490472	best: 4425.8490472 (6620)	total: 1m 33s	remaining: 5.32s
6621:	learn: 3232.7318981	test: 4424.6378953	best: 4424.6378953 (6621)	total: 1m 33s	remaining: 5.31s
6622:	learn: 3231.9302016	test: 4423.6709596	best: 4423.6709596 (6622)	total: 1m 33s	remaining: 5.29s
6623:	learn: 3231.3000491	test: 4423.5148169	best: 4423.5148169 (6623)	total: 1m 33s	remaining: 5.28s
6624:	learn: 3230.6385816	test: 4423.5304573	best: 4423.5148169 (6623)	total: 1m 33s	remaining: 5.27s
6625:	learn: 3229.3984791	test: 4422.5565186	best: 4422.5565186 (6625)	total: 1m 33s	remaining: 5.25s
6626:	learn: 3228.9174005	test: 4422.8086827	best: 4422.5565186 (6625)	total: 1m 3

6707:	learn: 3169.0222997	test: 4372.4399797	best: 4372.4399797 (6707)	total: 1m 34s	remaining: 4.11s
6708:	learn: 3167.8049332	test: 4371.6270337	best: 4371.6270337 (6708)	total: 1m 34s	remaining: 4.1s
6709:	learn: 3166.9116931	test: 4370.7814636	best: 4370.7814636 (6709)	total: 1m 34s	remaining: 4.08s
6710:	learn: 3166.3990883	test: 4369.7910911	best: 4369.7910911 (6710)	total: 1m 34s	remaining: 4.07s
6711:	learn: 3166.0485584	test: 4369.7613173	best: 4369.7613173 (6711)	total: 1m 34s	remaining: 4.05s
6712:	learn: 3165.5822856	test: 4369.5792373	best: 4369.5792373 (6712)	total: 1m 34s	remaining: 4.04s
6713:	learn: 3164.6116619	test: 4369.2676862	best: 4369.2676862 (6713)	total: 1m 34s	remaining: 4.03s
6714:	learn: 3164.2466164	test: 4369.0594217	best: 4369.0594217 (6714)	total: 1m 34s	remaining: 4.01s
6715:	learn: 3163.8953565	test: 4368.1383826	best: 4368.1383826 (6715)	total: 1m 34s	remaining: 4s
6716:	learn: 3163.5173716	test: 4368.0302397	best: 4368.0302397 (6716)	total: 1m 34s	r

6794:	learn: 3102.6217116	test: 4324.3349950	best: 4324.3349950 (6794)	total: 1m 35s	remaining: 2.89s
6795:	learn: 3101.7892196	test: 4323.5547561	best: 4323.5547561 (6795)	total: 1m 35s	remaining: 2.87s
6796:	learn: 3101.5408414	test: 4323.1973722	best: 4323.1973722 (6796)	total: 1m 35s	remaining: 2.86s
6797:	learn: 3100.0653088	test: 4322.6331298	best: 4322.6331298 (6797)	total: 1m 35s	remaining: 2.84s
6798:	learn: 3099.1512610	test: 4321.7741062	best: 4321.7741062 (6798)	total: 1m 35s	remaining: 2.83s
6799:	learn: 3098.4799478	test: 4321.4970821	best: 4321.4970821 (6799)	total: 1m 35s	remaining: 2.82s
6800:	learn: 3097.2805497	test: 4319.7214545	best: 4319.7214545 (6800)	total: 1m 35s	remaining: 2.8s
6801:	learn: 3096.2549784	test: 4319.5521645	best: 4319.5521645 (6801)	total: 1m 35s	remaining: 2.79s
6802:	learn: 3095.2660163	test: 4319.4755216	best: 4319.4755216 (6802)	total: 1m 35s	remaining: 2.77s
6803:	learn: 3094.1783936	test: 4319.2619740	best: 4319.2619740 (6803)	total: 1m 35

6876:	learn: 3040.7449818	test: 4283.6053137	best: 4283.6053137 (6876)	total: 1m 36s	remaining: 1.73s
6877:	learn: 3040.5260049	test: 4283.5220734	best: 4283.5220734 (6877)	total: 1m 36s	remaining: 1.72s
6878:	learn: 3040.4999070	test: 4283.4031879	best: 4283.4031879 (6878)	total: 1m 36s	remaining: 1.7s
6879:	learn: 3039.5986681	test: 4282.6903291	best: 4282.6903291 (6879)	total: 1m 36s	remaining: 1.69s
6880:	learn: 3038.2917206	test: 4281.6053372	best: 4281.6053372 (6880)	total: 1m 36s	remaining: 1.67s
6881:	learn: 3037.7037494	test: 4281.1967518	best: 4281.1967518 (6881)	total: 1m 36s	remaining: 1.66s
6882:	learn: 3036.6277086	test: 4280.5615270	best: 4280.5615270 (6882)	total: 1m 36s	remaining: 1.64s
6883:	learn: 3035.4143262	test: 4279.1978081	best: 4279.1978081 (6883)	total: 1m 36s	remaining: 1.63s
6884:	learn: 3035.1476011	test: 4278.6609149	best: 4278.6609149 (6884)	total: 1m 36s	remaining: 1.62s
6885:	learn: 3034.3899161	test: 4278.6862185	best: 4278.6609149 (6884)	total: 1m 36

6963:	learn: 2978.8247473	test: 4237.9120264	best: 4237.9120264 (6963)	total: 1m 37s	remaining: 505ms
6964:	learn: 2977.3864478	test: 4237.0535005	best: 4237.0535005 (6964)	total: 1m 37s	remaining: 491ms
6965:	learn: 2976.4008314	test: 4236.2682538	best: 4236.2682538 (6965)	total: 1m 37s	remaining: 477ms
6966:	learn: 2975.9649577	test: 4235.2343074	best: 4235.2343074 (6966)	total: 1m 37s	remaining: 463ms
6967:	learn: 2975.9524437	test: 4235.1793456	best: 4235.1793456 (6967)	total: 1m 37s	remaining: 449ms
6968:	learn: 2975.7673084	test: 4235.3066468	best: 4235.1793456 (6967)	total: 1m 37s	remaining: 435ms
6969:	learn: 2975.5216970	test: 4235.0856364	best: 4235.0856364 (6969)	total: 1m 37s	remaining: 421ms
6970:	learn: 2974.8445674	test: 4234.9914379	best: 4234.9914379 (6970)	total: 1m 37s	remaining: 407ms
6971:	learn: 2973.8085151	test: 4234.1148697	best: 4234.1148697 (6971)	total: 1m 37s	remaining: 393ms
6972:	learn: 2972.2518703	test: 4232.1387350	best: 4232.1387350 (6972)	total: 1m 3

In [60]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,total_square,7.063058
1,P1_F_max_distance_2_min,5.540566
2,P1_F_square_True_median,5.045235
3,street,3.288714
4,lng,3.133537
5,P1_F_simple_price_median,2.906380
6,F_square_True_max,2.590876
7,reform_mean_floor_count_500,2.464190
8,P1_F_square_False_median,2.260215
9,P1_F_square_True_min,2.032156


In [61]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 0.07633404577463415,
 'mdape': 0.04524162677013614,
 'rmse': 15587.357360602715,
 'r2': 0.9660787013474457,
 'raif_metric': 0.17329732837907535}

In [62]:
price0_preds = cat_model.predict(X_val)
metrics_stat(np.array(price0_preds)+1000, np.array(Y_val))

{'mape': 0.22629749264385002,
 'mdape': 0.19447444760686572,
 'rmse': 37925.78682309723,
 'r2': 0.6907862804815164,
 'raif_metric': 1.2450550091553048}

In [ ]:
{'mape': 0.32152263584077434,
 'mdape': 0.23299677152276008,
 'rmse': 87343.51408296668,
 'r2': -1.4305272953932735,
 'raif_metric': 1.9368490908547882}

In [ ]:
{'mape': 0.3938137450820849,
 'mdape': 0.27994944633239843,
 'rmse': 93738.73635776664,
 'r2': -3.7693751005637317,
 'raif_metric': 2.492385311141158}

{'mape': 0.28628533681632,
 'mdape': 0.21764848060697264,
 'rmse': 84390.73853846012,
 'r2': -1.6484418274393575,
 'raif_metric': 1.552534528099827}

In [ ]:
{'mape': 0.4006764913433361,
 'mdape': 0.28324961990050085,
 'rmse': 94343.39152646794,
 'r2': -4.286807955207906,
 'raif_metric': 2.6756861412533905}

{'mape': 0.2994072204832106,
 'mdape': 0.23136175216191418,
 'rmse': 85205.80509381538,
 'r2': -1.798582387807691,
 'raif_metric': 1.7313695017675064}

In [ ]:
{'mape': 0.45565440900238763,
 'mdape': 0.30249343647803606,
 'rmse': 100782.64525168539,
 'r2': -9.279126502012321,
 'raif_metric': 3.080882843995905}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [80]:
{'mape': 0.22629749264385002,
 'mdape': 0.19447444760686572,
 'rmse': 37925.78682309723,
 'r2': 0.6907862804815164,
 'raif_metric': 1.2450550091553048}
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds = cat_model.predict(test_data)+1000

In [81]:
price0_preds

array([49117.04277528, 41348.14916893, 41776.60274244, ...,
       57158.80289773, 50748.89571583, 59815.89997286])

In [84]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("overfit.csv", index=False)